# 预处理现代汉语词典、新华字典

In [1]:
from pathlib import Path
import pandas as pd
import re
import sqlite3


## 配置路径

In [2]:
# 配置数据集基本路径
folderpath_original_data_root = Path.cwd().parent.parent.parent.joinpath('EntelechyData/original')
Path.mkdir(folderpath_original_data_root, exist_ok=True)

folderpath_pretreat_data_root = Path.cwd().parent.parent.parent.joinpath('EntelechyData/pretreatment')
Path.mkdir(folderpath_pretreat_data_root, exist_ok=True)


## 预处理现代汉语词典

### 导入数据

导入 txt 文件格式的词典。

In [3]:
# 导入 txt 文件格式的词典
with open(Path(folderpath_original_data_root, '现代汉语词典/现代汉语词典.txt'), 'r', encoding='utf-8') as f:
    现代汉语词典_txt = f.readlines()


### 匹配字段

尝试匹配一些字段

In [4]:
# 匹配字段为 '   word' 的行
reg_字段word = re.compile(r'^   word : ')
lines_字段word = [i for i in 现代汉语词典_txt if reg_字段word.match(i)]
lines_字段word = [reg_字段word.sub('', i).strip() for i in lines_字段word]  # 去掉 '   word : ' 字段

In [5]:
# 匹配字段为 'content' 的行
reg_字段content = re.compile(r'^content : ')
lines_字段content = [i for i in 现代汉语词典_txt if reg_字段content.match(i)]
lines_字段content = [reg_字段content.sub('', i).strip() for i in lines_字段content]  # 去掉 'content : ' 字段

In [6]:
# 匹配字段为 '    ord : ' 的行
reg_字段ord = re.compile(r'^    ord : ')
lines_字段ord = [i for i in 现代汉语词典_txt if reg_字段ord.search(i)]
lines_字段ord = [reg_字段ord.sub('', i).strip() for i in lines_字段ord]  # 去掉 '    ord : ' 字段

In [7]:
# 匹配字段为 '  spell : ' 的行
reg_字段spell = re.compile(r'^  spell : ')
lines_字段spell = [i for i in 现代汉语词典_txt if reg_字段spell.search(i)]
lines_字段spell = [reg_字段spell.sub('', i).strip() for i in lines_字段spell]  # 去掉 '  spell : ' 字段

联合上述字段，生成一个 pandas DataFrame。

In [8]:
# 生成一个 pandas DataFrame
df_现代汉语词典 = pd.DataFrame(columns=['word', 'content', 'ord', 'spell'])
df_现代汉语词典['word'] = lines_字段word
df_现代汉语词典['content'] = lines_字段content
df_现代汉语词典['ord'] = lines_字段ord
df_现代汉语词典['spell'] = lines_字段spell
# 添加 'id_现代汉语词典' 字段，作为主键，排到第一列
df_现代汉语词典['id_现代汉语词典'] = range(1, len(df_现代汉语词典) + 1)
df_现代汉语词典 = df_现代汉语词典[['id_现代汉语词典', 'word', 'content', 'ord', 'spell']]

查找词。

In [10]:
df_现代汉语词典[df_现代汉语词典['word'] == '打']

,id_现代汉语词典,word,content,ord,spell


### 导出数据

导出为 pkl 文件、sqlite 数据库。

In [11]:
folderpath_pretreat_现代汉语词典 = Path(folderpath_pretreat_data_root, '现代汉语词典')
Path.mkdir(folderpath_pretreat_现代汉语词典, exist_ok=True)

if not Path(folderpath_pretreat_现代汉语词典, '现代汉语词典.pkl').exists():
    df_现代汉语词典.to_pickle(Path(folderpath_pretreat_现代汉语词典, '现代汉语词典.pkl'))  # 导出为 pkl 文件

if not Path(folderpath_pretreat_现代汉语词典, '现代汉语词典.db').exists():
    conn = sqlite3.connect(Path(folderpath_pretreat_现代汉语词典, '现代汉语词典.db'))  # 导出为 sqlite 数据库

## 预处理新华字典

### 导入数据

导入 txt 文件格式的词典。

In [12]:
# 导入 json 文件格式的词典
import json

新华字典_word_json = json.loads(Path(folderpath_original_data_root, '新华字典/word.json').read_text())

### 匹配字段

尝试匹配一些字段

In [13]:
# 匹配字段为 '   word' 的行
reg_字段word = re.compile(r'^   word : ')
lines_字段word = [i for i in 现代汉语词典_txt if reg_字段word.match(i)]
lines_字段word = [reg_字段word.sub('', i).strip() for i in lines_字段word]  # 去掉 '   word : ' 字段

### 转换为 DataFrame 


In [14]:
columns_新华字典 = 新华字典_word_json[0].keys()
df_新华字典 = pd.DataFrame(columns=columns_新华字典)
for i, item in enumerate(新华字典_word_json):
    df_新华字典.loc[i] = list(item.values())

### 导出数据

导出为 pkl 文件、sqlite 数据库。

In [15]:
folderpath_pretreat_新华字典 = Path(folderpath_pretreat_data_root, f'新华字典')
Path.mkdir(folderpath_pretreat_新华字典, exist_ok=True)

if not Path(folderpath_pretreat_新华字典, '新华字典.pkl').exists():
    df_新华字典.to_pickle(Path(folderpath_pretreat_新华字典, '新华字典.pkl'))  # 导出为 pkl 文件

if not Path(folderpath_pretreat_新华字典, '新华字典.db').exists():
    conn = sqlite3.connect(Path(folderpath_pretreat_新华字典, '新华字典.db'))  # 导出为 sqlite 数据库